In [1]:
import os
import numpy as np
import nibabel as nib

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Activation, LeakyReLU
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2

import matplotlib.pyplot as plt
import gc
import seaborn as sns

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image
import tempfile
from math import ceil

In [2]:
# FUNÇÕES
# Winsorize -> reduz outliers, limitando os percentis inf e sup
def winsorize_image(image_data, lower_percentile=1, upper_percentile=99): #reduz valores extremos
    lower_bound = np.percentile(image_data, lower_percentile)
    upper_bound = np.percentile(image_data, upper_percentile)
    winsorized_data = np.clip(image_data, lower_bound, upper_bound)
    return winsorized_data

# Normalization -> valores de voxels entre 0 e 1
def normalize_image_min(image_data): 
    min_val = np.min(image_data)
    max_val = np.max(image_data)
    normalized_data = (image_data - min_val) / (max_val - min_val)
    return normalized_data

# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def load_nifti_paths(base_dir, class_names):
    image_paths = []
    labels = []
    
    # Caminhos das subpastas
    for label in class_names:
        label_dir = os.path.join(base_dir, label)
        for fname in os.listdir(label_dir):
            img_path = os.path.join(label_dir, fname)
            image_paths.append(img_path)
            labels.append(label)

    # Codificando os rótulos
    label_encoder = LabelEncoder()

    # Inverter a ordem das classes explicitamente
    label_encoder.classes_ = np.array(class_names)

    # Convertendo a lista de rótulos para um array NumPy
    labels_array = np.array(labels)

    # Codificando os rótulos (agora 'cn' será 0 e 'ad' será 1)
    labels_encoded = label_encoder.transform(labels_array)

    # Transformando os rótulos para one-hot encoding
    labels_one_hot = to_categorical(labels_encoded, num_classes=len(class_names))

    # Embaralhar os dados
    image_paths, labels_one_hot = shuffle(image_paths, labels_one_hot, random_state=42)

    return image_paths, labels_one_hot, label_encoder.classes_

# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def nifti_data_generator_3d(image_paths, labels, batch_size, shuffle_each_epoch=False):
    while True:
        if shuffle_each_epoch:
            image_paths, labels = shuffle(image_paths, labels, random_state=42)

        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            images = []
            
            for path in batch_paths:
                # Carregar a imagem NIfTI e garantir o formato correto
                img = nib.load(path).get_fdata()  # Shape original: 
                img = img[..., np.newaxis]       # Adicionar a dimensão do canal: 
                images.append(img)
            
            # Converter lista para array NumPy e garantir o shape correto
            images = np.array(images) 
            batch_labels = np.array(batch_labels)

            # Liberar memória
            gc.collect()
            
            yield images, batch_labels


# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def nifti_data_generator_pdf(image_paths, labels, batch_size, shuffle_each_epoch=False):
    while True:
        if shuffle_each_epoch == True:
            image_paths, labels = shuffle(image_paths, labels, random_state=42)

        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            images = []
            
            for path in batch_paths:
                img = nib.load(path).get_fdata()
                images.append(img)
            
            images = np.array(images).reshape((-1, *img.shape, 1))
            batch_labels = np.array(batch_labels)

            gc.collect()
            
            yield images, batch_labels, batch_paths

def plot_training_history(history, dir):
    plt.figure(figsize=(12, 4))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Graphic')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['categorical_accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Graphic')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.savefig(os.path.join(dir, 'training_history.png'))

    plt.show()

def plot_confusion_matrix(y_true, y_pred, dir):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.savefig(os.path.join(dir, 'confusion_matrix.png'))
    plt.show()

def create_model_3d(input_shape, n_classes):
    model = Sequential([        
        Input(shape=input_shape),  # Formato de entrada: (1, 145, 182, 7)

        # Camada 1 - Filtro 3x3
        Conv3D(32, (3, 3, 3), padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.3),  
        MaxPooling3D(pool_size=(1, 2, 2), padding='same'),
        Dropout(0.4),

        # Camada 2 - Filtro 5x5
        Conv3D(64, (3, 3, 3), padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.3),  
        MaxPooling3D(pool_size=(1, 2, 2), padding='same'),
        Dropout(0.4),

        # Camada 3 - Filtro 3x3
        Conv3D(128, (3, 3, 3), padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.3),  
        MaxPooling3D(pool_size=(2, 2, 2), padding='same'),
        Dropout(0.4),

        # Camada 4 - Filtro 5x5
        Conv3D(256, (3, 3, 3), padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.3),  
        MaxPooling3D(pool_size=(1, 2, 2), padding='same'), # talvez reduzir mais uma vez
        Dropout(0.4),

        # Camada de saída convolucional
        Flatten(),

        # Camadas densas
        Dense(256, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        LeakyReLU(negative_slope=0.3),  

        Dense(128, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        LeakyReLU(negative_slope=0.3),  

        # Camada de saída
        Dense(n_classes, activation='softmax')  # Sem regularização L2 na camada de saída
    ])
    
    return model

In [4]:
# Definindo caminhos
dir_base = r"C:\Users\Team Taiane\Desktop\ADNI\FULL_ADNI\processed_7_slices_data\7_slices_axial"

train_dir = f'{dir_base}/train'
val_dir = f'{dir_base}/validation'
test_dir = f'{dir_base}/test'
results_dir = f'{dir_base}/results/3d'

# Criar o diretório de resultados se ele não existir
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Nome das classes
class_names = ['cn', 'emci', 'mci', 'lmci', 'ad']

n_classes = len(class_names)

train_image_paths, train_labels, class_labels = load_nifti_paths(train_dir, class_names)
val_image_paths, val_labels, _ = load_nifti_paths(val_dir, class_names)
test_image_paths, test_labels, _ = load_nifti_paths(test_dir, class_names)

print(f"N treino: {len(train_image_paths)}")
print(f"N validation: {len(val_image_paths)}")
print(f"N validation: {len(test_image_paths)}")

batch_size = 64

steps_per_epoch = len(train_image_paths) // batch_size
validation_steps = len(val_image_paths) // batch_size

train_generator = nifti_data_generator_3d(train_image_paths, train_labels, batch_size, shuffle_each_epoch=False)
val_generator = nifti_data_generator_3d(val_image_paths, val_labels, batch_size)
test_generator = nifti_data_generator_3d(test_image_paths, val_labels, batch_size)

In [ ]:
n = len(os.listdir(results_dir))

if (n > 0):
    if (len(os.listdir(os.path.join(results_dir, f'test_{n}'))) < 3): 
        for item in os.listdir(os.path.join(results_dir, f"test_{n}")):
            os.remove(os.path.join(results_dir,  f"test_{n}", item))
        os.removedirs(os.path.join(results_dir, f'test_{n}'))
        n -= 1

folder_name = f"test_{str(n+1)}"
results_dir = os.path.join(results_dir, folder_name)
os.makedirs(results_dir, exist_ok=True)
print(f"pasta {folder_name} criada")

In [ ]:
# Compila model
input_shape_x = (7, 182, 155, 1) # sagital
input_shape_y = (145, 7, 155, 1) # coronal
input_shape_z = (145, 182, 7, 1) # axial

model = create_model_3d(input_shape_x, n_classes)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

In [7]:
epochs = 120

new_model_name_ker = (f"binary_classifier_{epochs}_epochs_batch_{batch_size}_{n_classes}_classes.keras")

# Supondo que você já tenha um modelo e esteja treinando com `fit`
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=15,                 
    verbose=1
)

# Defina o nome do arquivo para salvar o melhor modelo
model_checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(results_dir, new_model_name_ker),    
    monitor='val_categorical_accuracy',
    save_best_only=True, 
    mode='max', 
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

log_path = os.path.join(results_dir, 'log_treino.csv')

csv_log = CSVLogger(log_path, append=False)

In [ ]:
print(f"Iniciando treinamento do modelo {new_model_name_ker} para classes {class_names}")

# Treinamento
history = model.fit(
    train_generator,
    epochs=epochs,
    verbose=1,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint_callback, reduce_lr, csv_log]
)

In [ ]:
val_pdf_generator = nifti_data_generator_pdf(val_image_paths, val_labels, batch_size, shuffle_each_epoch=False)

y_true = []
y_pred = []
y_image = []
y_paths = []

best_model = load_model(os.path.join(results_dir, new_model_name_ker))

for i in range(0, ceil(len(val_image_paths) / batch_size)):
    batch_images, batch_labels, batch_paths = next(val_pdf_generator)
    y_image.append(batch_images)
    y_true.append(batch_labels)
    y_paths.append(batch_paths)
    
    # Fazendo predição para o lote atual
    batch_pred = best_model.predict(batch_images)
    y_pred.append(batch_pred)

# Concatenando as predições e os rótulos verdadeiros
y_paths = np.concatenate(y_paths)
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
y_image = np.concatenate(y_image)

# Convertendo as predições para rótulos (a classe com maior probabilidade)
y_true_labels = np.argmax(y_true, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
# Função para carregar uma imagem NIfTI e extrair uma fatia específica do eixo Z
def load_nifti_image_pdf(file_path):
    img = nib.load(file_path) 
    data = img.get_fdata()  
    slice_2d = data[2, :, :]

    # implementar lógica de retornar vetor com cada fatia como imagem única

    return slice_2d

# Função para criar o PDF
def create_pdf(y_paths, y_true_labels, y_pred_labels, output_pdf_path):
    c = canvas.Canvas(output_pdf_path, pagesize=letter)
    width, height = letter  # Dimensões da página no PDF

    i = 0
    for item in y_paths:
        true = ''
        pred = ''
        # Carregar a imagem NIfTI e obter a fatia 2D no eixo Z
        nifti_image = load_nifti_image_pdf(item)
        
        # Converter a fatia 2D para uma imagem 8-bit (grayscale) para visualização
        img = Image.fromarray(np.uint8(nifti_image / np.max(nifti_image) * 255))  # Normalizar e converter
        img = img.convert("RGB")  # Garantir que a imagem tenha 3 canais (RGB)

        # Redimensionar a imagem para se ajustar ao tamanho da página
        img_width, img_height = img.size
        aspect_ratio = img_height / float(img_width)
        new_width = width * 0.2  # Definir largura como 80% da largura da página
        new_height = new_width * aspect_ratio
        img = img.resize((int(new_width), int(new_height)))

        # Criar um arquivo temporário para salvar a imagem
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_file:
            temp_file_path = temp_file.name
            img.save(temp_file_path)

        # configurar para printar as 7 fatias em uma página inteira, com as informações de label predito e esperado

        # Colocar a imagem no PDF usando o caminho temporário
        if i % 12 < 4:
            x = 80
        elif i % 12 < 8:
            x = width - 2.35*new_width - 80
        else:
            x = width - new_width - 80

        y = height - (new_height + 80)*((i%4)+1)

        c.drawImage(temp_file_path, x, y, width=new_width, height=new_height)

        # Escrever os rótulos
        true_label = y_true_labels[i]
        pred_label = y_pred_labels[i]

        #ver como transformar os labels de maneira inteligente
        true = class_names[true_label]
        pred = class_names[pred_label]

        # Definir a cor para os rótulos
        if true_label == pred_label:
            pred_color = (0, 1, 0)  # Verde
        else:
            pred_color = (1, 0, 0)  # Vermelho
        
        #Nome paciente (em preto)
        c.setFont("Helvetica", 12)
        c.setFillColorRGB(0, 0, 0)  # Preto
        c.drawString(x+24, y+new_height+50, f"{os.path.basename(item)}")

        # Rótulo esperado (em preto)
        c.setFont("Helvetica", 12)
        c.setFillColorRGB(0, 0, 0)  # Preto
        c.drawString(x+26, y+new_height+35, f"Expected: {true}")

        # Rótulo predito
        c.setFont("Helvetica", 12)
        c.setFillColorRGB(*pred_color)  # Verde ou Vermelho
        c.drawString(x+26, y+new_height+20, f"Predicted: {pred}")

        # Rótulo predito
        c.setFont("Helvetica", 12)
        c.setFillColorRGB(*pred_color)  # Verde ou Vermelho
        c.drawString(x+26, y+new_height+5, f"Prob: {max(y_pred[i])*100:.2f}%")

        # Avançar para a próxima imagem
        i += 1
        
        # Adicionar uma nova página no PDF a cada 2 imagens (se necessário)
        if i % 12 == 0:  # Por exemplo, a cada 2 imagens, adicionamos uma nova página
            c.showPage()

    # Salvar o PDF
    c.save()

# Exemplo de uso:
val_pdf_path = os.path.join(results_dir, "validation_predictions.pdf")
create_pdf(y_paths, y_true_labels, y_pred_labels, val_pdf_path)

In [ ]:
# Gerar relatório
report = classification_report(y_true_labels, y_pred_labels)
print(report)

# Escrevendo o relatório em um arquivo .txt
with open(os.path.join(results_dir, "classification_report.txt"), "w") as file:
    file.write(report)

# Calcular a matriz de confusão
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Plotando a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Previsões')
plt.ylabel('Valores Reais')
plt.title('Matriz de Confusão')
plt.savefig(f'{results_dir}/confusion_matrix.png')
plt.show()

# Plotando o histórico de treinamento após o treinamento
plot_training_history(history, results_dir)

In [ ]:
best_model = load_model(r"C:\Users\Team Taiane\Desktop\ADNI\FULL_ADNI\processed_7_slices_data\7_slices_axial\results\3d\axial_certo\test_5\binary_classifier_120_epochs_batch_64_5_classes.keras")

test_pdf_generator = nifti_data_generator_pdf(test_image_paths, test_labels, batch_size, shuffle_each_epoch=False)

y_true = []
y_pred = []
y_image = []
y_paths = []

for i in range(0, ceil(len(test_image_paths) / batch_size)):
    batch_images, batch_labels, batch_paths = next(test_pdf_generator)
    y_image.append(batch_images)
    y_true.append(batch_labels)
    y_paths.append(batch_paths)
    
    # Fazendo predição para o lote atual
    batch_pred = best_model.predict(batch_images)
    y_pred.append(batch_pred)

# Concatenando as predições e os rótulos verdadeiros
y_paths = np.concatenate(y_paths)
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
y_image = np.concatenate(y_image)

# Convertendo as predições para rótulos (a classe com maior probabilidade)
y_true_labels = np.argmax(y_true, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
print(f"Tamanho de y_true_labels: {len(y_true_labels)}")
print(f"Tamanho de y_pred_labels: {len(y_pred_labels)}")

In [ ]:
# Gerar relatório
report = classification_report(y_true_labels, y_pred_labels)
print(report)

# Escrevendo o relatório em um arquivo .txt
with open(os.path.join(results_dir, "classification_report.txt"), "w") as file:
    file.write(report)

# Calcular a matriz de confusão
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Plotando a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Previsões')
plt.ylabel('Valores Reais')
plt.title('Matriz de Confusão')
plt.savefig(f'{results_dir}/confusion_matrix.png')
plt.show()